In [5]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

HOST = "milvus"
PORT = "19530"
COLLECTION_NAME = 'TextSearchCollection'
INDEX_PARAMS = {
    'metric_type': 'COSINE',
    'index_type': 'IVF_FLAT',
    'params': {'nlist': 4},
}
DIMENSTION = 1536 # OpenAI Standard 
 
connections.connect(host=HOST, port=PORT)

In [6]:
def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='id', dtype=DataType.INT64, descrition='id', is_primary=True, auto_id=True),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=DIMENSTION),
        FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=4096)
    ]
    schema = CollectionSchema(fields=fields)
    collection = Collection(name=collection_name, schema=schema)

    collection.create_index(field_name="embedding", index_params=INDEX_PARAMS)
    return collection

collection = create_milvus_collection(COLLECTION_NAME, DIMENSTION)
collection.load() # load 完之後才可以搜尋

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain_openai import OpenAIEmbeddings 

def handle_text(text: str):
    CHUNK_SIZE = 400
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_SIZE//5)

    return [Document(page_content=x) for x in text_splitter.split_text(text)]

def parse_content(text):
    documents = []
    documents = handle_text(text)
    embedding_model = OpenAIEmbeddings()
    return [
        {
            'embedding': embedding_model.embed_documents([d.page_content])[0],
            'content': d.page_content,
        } for d in documents
    ]

with open('./articles/天劍一夢.txt', 'r') as file:
    text = file.read()
    # print(text)
    entities = parse_content(text)
    print(entities[0])
    collection.insert(entities)

{'embedding': [-0.009441361221505724, -0.031747149346794644, -0.0003862039081145105, -0.007227207585490465, -0.009888133398750923, 0.012693604658259496, -0.018909002501915857, -0.011221882244066726, -0.025597455375689387, -0.020078495856805236, 0.010394038036837347, 0.006419074353761914, 0.00798934931715252, 0.007030101555772274, 0.0021714473761424427, 0.0060708542600532885, 0.008968307122711072, -0.011261303263745859, 0.005101751243753259, -0.008449262455172446, -0.0005867999525657415, 0.016767120890532766, -0.017713229088122066, -0.010058958438242185, -0.014283590131489669, 0.03432266589993829, 0.01919809060044452, -0.010571433556715972, 0.013600289973524622, -0.0092639657016271, 0.005564949621966866, 0.009434790741118361, -0.029592128637281865, 0.005289001552890406, 0.001117753382975833, 0.009625326290449189, 0.006461780613634729, -0.024730189217458353, 0.032036239307968346, -0.01467780125960353, 0.023652678862701965, -0.018317685344083803, 0.0024868164648980365, 0.01817314222614199

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

# ===== Step 1: Embeding ===== #
query = '天劍門歷史？'
query_embedding = [OpenAIEmbeddings().embed_query(query)]


# ===== Step 2: Search knowledge ===== #
search_result = collection.search(
    data=query_embedding,
    anns_field = 'embedding',
    limit=5, 
    param={
        'metric_type': 'COSINE', 
        'params': {},
    },
    output_fields=['id', 'content'],
)

for hits in search_result:
    for hit in hits:
        print(hit.get('id'), hit.get('content'))


# ====== Step 3: provide context to LLM ===== #
context = '\n'.join(hit.get('content') for hit in hits for hits in search_result)
parser = StrOutputParser()
prompt = PromptTemplate(
    template='請根據以下資料回答使用者問題，只能依據資料回答，不得捏造. 問題:{query}. 資料:{context}',
    input_variables=['query', 'context'],
    
)
llm = ChatOpenAI(model='gpt-3.5-turbo')

chain = prompt | llm | parser
response = chain.invoke({'query': query, 'context': context})
print(response)


448711390431806728 在遼闊的中原，有一座名為「天劍門」的武林門派，以其絕世武功和正義之名遠播。
故事發生在亂世之末，江湖上群雄並起，爭鬥不斷。
天劍門因其崇尚正義，經常介入江湖紛爭，調解各大門派的恩怨情仇。
然而，一場突如其來的浩劫將天劍門推向了風暴的中心。
門中高手陣亡無數，唯留下傳說中的絕世劍譜遺失於江湖。
天下武林為此劍譜再起波瀾，各方勢力暗流湧動，江湖再度陷入腥風血雨之中。
在這風雲變幻的世界裡，一個年輕的劍客即將踏上尋找絕世劍譜、繼承天劍門遺志的征途。
448711390431806732 故事最終進入高潮，蕭雲和他的同伴們面對著前所未有的挑戰。
古老門派的首領揭露了自己的野心，他欲以絕世劍譜中隱藏的武學之道，征服整個武林。
在一場決定性的戰鬥中，蕭雲與柳夢璃攜手，利用他們對絕世劍譜的深刻理解和自身的武學實力，與邪惡勢力進行了生死對決。
經過一番激烈的戰鬥，正義終於戰勝邪惡。蕭雲卻也重傷昏迷。


蕭雲夢回少年時代，他夢見自己在天劍門的日子，與師兄弟們揮劍練武，夢中的一切都是如此和平與美好。
當他醒來，發現自己躺在柳夢璃的小屋中，外面的世界依舊紛亂不息，江湖的爭鬥似乎從未停止。
蕭雲深刻領悟到，無論外界如何變幻，真正的武功與智慧不僅是征服敵手，更是征服自己內心的恐懼與迷惑。
他明白了黃粱一夢的意義——人生短暫，如夢一般，真正重要的是堅守內心的信念與正義，無悔於自己的選擇。
448711390431806731 在茫茫江湖之中，蕭雲不僅要尋找失散的絕世劍譜，還必須面對各種勢力的考驗與誘惑。
在一次偶然的機會中，他救下了一名身世神秘的女子，名叫柳夢璃。
柳夢璃似乎對絕世劍譜有著某種特殊的聯繫，但她過去的記憶卻支離破碎。
蕭雲與柳夢璃之間逐漸建立起深厚的信任和情感，柳夢璃也逐漸成為了蕭雲旅途中不可或缺的伙伴。
然而，隨著二人越來越接近絕世劍譜的秘密，柳夢璃的真實身份也逐漸浮出水面，與之相關的陰謀和秘密開始緩緩解開。


隨著劇情的推進，蕭雲發現江湖上的各大門派不僅為了絕世劍譜而相互爭鬥，背後更有一股神秘的力量在操縱這一切。
這股力量來自一個被遺忘的古老門派，他們暗中收集各種武學秘籍，企圖破解武學之極，掌控整個武林。
蕭雲在逐步揭開真相的過程中，也進一步提升了自己的武學修為，從一個熱血青年成長為真正的武林高手。
他開始理解，真正的強大不僅是武功之